In [43]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [44]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/var/folders/z6/0v89xnrd4vgcf52w_f36js500000gn/T/ipykernel_69646/2708364295.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [45]:
data = pd.read_csv("throwdowns_transactions.csv", index_col='Player Name')
data

,Set,Series,Play,Date of Moment,Team,Circulation Count,Serial,Purchase Price,Buyer,Seller,Transaction Date,Top Shot Debut,Rookie Premiere,Rookie Mint,Rookie Year,Edition State,Play ID,Set ID,Top Shot Link
Player Name,,,,,,,,,,,,,,,,,,,
Blake Griffin,Throwdowns,2,Dunk,2021-04-07 23:30:00+00:00,Brooklyn Nets,382,253,265,molehero,Moizoos,2021-05-17 18:07:47+00:00,False,False,False,False,CHALLENGE_REWARD_RELEASED,f3140cd1-8f9e-4483-853d-9bce3d63180d,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+f3140cd1-8f9e-4483-853d-9bce3d63180d
Blake Griffin,Throwdowns,2,Dunk,2021-04-07 23:30:00+00:00,Brooklyn Nets,382,228,273,Cowboyfan95,astudlymuffin,2021-05-17 18:05:49+00:00,False,False,False,False,CHALLENGE_REWARD_RELEASED,f3140cd1-8f9e-4483-853d-9bce3d63180d,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+f3140cd1-8f9e-4483-853d-9bce3d63180d
Blake Griffin,Throwdowns,2,Dunk,2021-04-07 23:30:00+00:00,Brooklyn Nets,382,283,275,benjamin23,Zenmonkey,2021-05-17 17:58:55+00:00,False,False,False,False,CHALLENGE_REWARD_RELEASED,f3140cd1-8f9e-4483-853d-9bce3d63180d,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+f3140cd1-8f9e-4483-853d-9bce3d63180d
Blake Griffin,Throwdowns,2,Dunk,2021-04-07 23:30:00+00:00,Brooklyn Nets,382,359,285,FatTom23,LBJ1,2021-05-17 17:50:05+00:00,False,False,False,False,CHALLENGE_REWARD_RELEASED,f3140cd1-8f9e-4483-853d-9bce3d63180d,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+f3140cd1-8f9e-4483-853d-9bce3d63180d
Blake Griffin,Throwdowns,2,Dunk,2021-04-07 23:30:00+00:00,Brooklyn Nets,382,79,348,justsaiyan,knowing_fur_seal8672,2021-05-17 17:45:53+00:00,False,False,False,False,CHALLENGE_REWARD_RELEASED,f3140cd1-8f9e-4483-853d-9bce3d63180d,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+f3140cd1-8f9e-4483-853d-9bce3d63180d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Donovan Mitchell,Throwdowns,2,Dunk,2021-01-18 01:00:00+00:00,Utah Jazz,809,719,230,winterhawk,JPo,2022-02-07 02:45:54+00:00,False,False,False,False,VISIBLE,05a197a7-4427-4ac0-8115-effbf9e7a5c1,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+05a197a7-4427-4ac0-8115-effbf9e7a5c1
Donovan Mitchell,Throwdowns,2,Dunk,2021-01-18 01:00:00+00:00,Utah Jazz,809,591,247,Nacho_Landon,Mattes1988,2022-02-07 18:41:09+00:00,False,False,False,False,VISIBLE,05a197a7-4427-4ac0-8115-effbf9e7a5c1,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+05a197a7-4427-4ac0-8115-effbf9e7a5c1
Donovan Mitchell,Throwdowns,2,Dunk,2021-01-18 01:00:00+00:00,Utah Jazz,809,393,225,sweltering_river_dolphin6508,ktop34,2022-02-08 20:48:24+00:00,False,False,False,False,VISIBLE,05a197a7-4427-4ac0-8115-effbf9e7a5c1,757f23fd-f7ae-465c-a006-f09dcfd5dbd5,https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+05a197a7-4427-4ac0-8115-effbf9e7a5c1


In [46]:
player_name = data.index
player_name

Index(['Blake Griffin', 'Blake Griffin', 'Blake Griffin', 'Blake Griffin', 'Blake Griffin', 'Blake Griffin', 'Blake Griffin', 'Blake Griffin', 'Blake Griffin', 'Blake Griffin',
       ...
       'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell', 'Donovan Mitchell'], dtype='object', name='Player Name', length=107963)

In [47]:
data['unique_id'] = data['Transaction Date'].astype(str) + data['Play ID'].astype(str)

In [48]:
data.set_index('unique_id', inplace=True)


In [49]:
data.reset_index(inplace=True)

In [50]:
data.dtypes

unique_id            object
Set                  object
Series               int64 
Play                 object
Date of Moment       object
Team                 object
Circulation Count    int64 
Serial               int64 
Purchase Price       int64 
Buyer                object
Seller               object
Transaction Date     object
Top Shot Debut       bool  
Rookie Premiere      bool  
Rookie Mint          bool  
Rookie Year          bool  
Edition State        object
Play ID              object
Set ID               object
Top Shot Link        object
dtype: object

In [51]:
def get_null_rate(df, col_name):
    """
    Returns the null rate in a column of a dataframe.
    """
    null_rate = len(df[df[col_name].isna()]) / len(df[col_name]) * 100
    return null_rate

In [52]:
def get_null_rates_df(df):
    """
    Outputs a dataframe with the null rates of each of the columns in the input dataframe.
    """
    variables = list(df.columns)[1:]

    null_rates = []
    for var in variables:
        null_rate = get_null_rate(df, var)
        null_rates.append(null_rate)

    all_null_rates_df = pd.DataFrame({'Variable': variables, 'Null Rate': null_rates}).sort_values(by='Null Rate',
                                                                                                   ascending=False)

    return all_null_rates_df

In [53]:
def get_column_categorization_df(input_df): 
    col_types = []
    for col_name in input_df.columns: 
        col_types.append((input_df[col_name].dtypes.name, col_name))

    col_types_df = pd.DataFrame({'col_type':[],'col_name':[]})

    for col_type in col_types: 
        temp_df = pd.DataFrame({'col_type':[col_type[0]],'col_name':[col_type[1]]})
        col_types_df = col_types_df.append(temp_df)
    col_types_df.sort_values(by=['col_type', 'col_name'])

    return col_types_df



In [54]:
col_types_df = get_column_categorization_df(data)
col_types_df

,col_type,col_name
0,object,unique_id
0,object,Set
0,int64,Series
0,object,Play
0,object,Date of Moment
0,object,Team
0,int64,Circulation Count
0,int64,Serial
0,int64,Purchase Price
0,object,Buyer


In [55]:
bool_variables = list(col_types_df[col_types_df['col_type']=='bool'].col_name)
bool_variables

['Top Shot Debut', 'Rookie Premiere', 'Rookie Mint', 'Rookie Year']

In [56]:
# convert bool to numericals
for var in bool_variables:
    data[var] = data[var].astype(int)

In [57]:
potential_important_vars = list(col_types_df[col_types_df['col_type']=='object'].col_name)
potential_important_vars.remove('Transaction Date')
potential_important_vars.remove('Date of Moment')
potential_important_vars

['unique_id',
 'Set',
 'Play',
 'Team',
 'Buyer',
 'Seller',
 'Edition State',
 'Play ID',
 'Set ID',
 'Top Shot Link']

In [58]:
def get_column_categorization_df(input_df): 
    col_types = []
    for col_name in input_df.columns: 
        col_types.append((input_df[col_name].dtypes.name, col_name))

    col_types_df = pd.DataFrame({'col_type':[],'col_name':[]})

    for col_type in col_types: 
        temp_df = pd.DataFrame({'col_type':[col_type[0]],'col_name':[col_type[1]]})
        col_types_df = col_types_df.append(temp_df)
    col_types_df.sort_values(by=['col_type', 'col_name'])

    return col_types_df

In [59]:
def get_categorical_rate_df(df, var_name):

    col_name = df.groupby([var_name]).count().columns[0]

    temp_df = df.groupby([var_name])[[col_name]].count()
    temp_df['total_scans'] = temp_df[col_name].sum()
    temp_df['pct_over_total'] = temp_df[col_name] / temp_df['total_scans']

    temp_df.rename(columns={temp_df.columns[0]: "num_entries"}, inplace=True)

    # col_names = list(temp_df.columns)
    # col_names[0] = 'num_entries'
    # temp_df.columns = col_names

    return temp_df

def get_important_fields_df(df, var_name, threshold=100):


    temp_df = df.groupby([var_name])[['unique_id']].count()
    temp_df2 = temp_df[temp_df['unique_id'] > threshold]

    main_values = list(temp_df2.index)

    return df[df[var_name].isin(main_values)]

def get_correlation_df(df, var_name, target_var='Purchase Price', min_correlation=0.05):

    df2 = pd.get_dummies(df[[var_name, target_var]], columns=[var_name])

    # x_labels = []
    # labels = list(df2.columns[1:])

    # for i in range(0, len(labels)):
    #     x_labels.append(labels[i].split('_')[-1])

    df3 = df2.corrwith(df2[target_var])[1:].to_frame()
    df3.columns = ['Correlation with {}'.format(target_var)]

    df3 = df3[np.abs(df3['Correlation with {}'.format(target_var)]) > min_correlation]
    df3['var_name'] = var_name

    values = list(df3.index)

    df3['values'] = [value.replace(var_name + "_", "") for value in values]

    return df3

def get_most_important_variables(df, potential_important_vars, threshold_count=20, min_correlation=0.05,
                                 target_var='Purchase Price'):

    var_name = potential_important_vars[0]
    temp_df = get_important_fields_df(df, var_name, threshold=threshold_count)

    high_corr_df = get_correlation_df(temp_df, var_name, target_var=target_var,
                                           min_correlation=min_correlation)

    for i in range(1, len(potential_important_vars)):
        var_name = potential_important_vars[i]
        reduced_df = get_important_fields_df(df, var_name, threshold=threshold_count)
        high_corr_temp_df = get_correlation_df(reduced_df, var_name, target_var=target_var,
                                                    min_correlation=min_correlation)
        high_corr_df = high_corr_df.append(high_corr_temp_df)
        print(var_name)
        print(high_corr_temp_df)

    return high_corr_df

In [60]:
high_corr_df = get_most_important_variables(data, potential_important_vars, threshold_count=20, min_correlation=0.05,
                                 target_var='Purchase Price')

Set
Empty DataFrame
Columns: [Correlation with Purchase Price, var_name, values]
Index: []
Play
Empty DataFrame
Columns: [Correlation with Purchase Price, var_name, values]
Index: []
Team
                             Correlation with Purchase Price var_name                  values
Team_Chicago Bulls           0.062063                         Team     Chicago Bulls         
Team_Denver Nuggets         -0.058308                         Team     Denver Nuggets        
Team_Detroit Pistons         0.058498                         Team     Detroit Pistons       
Team_Minnesota Timberwolves  0.087512                         Team     Minnesota Timberwolves
Team_Portland Trail Blazers  0.184285                         Team     Portland Trail Blazers
Team_Sacramento Kings       -0.059444                         Team     Sacramento Kings      
Team_Utah Jazz               0.058228                         Team     Utah Jazz             
Buyer
                    Correlation with Purchase Price va

In [61]:
for i in range(0, len(high_corr_df)): 
        new_var_name = high_corr_df.iloc[[i]].index[0]
        var_name = high_corr_df.iloc[[i]]['var_name'].iloc[0]
        value = high_corr_df.iloc[[i]]['values'].iloc[0]
        data[new_var_name] = np.where(data[var_name]==value,1,0)

In [62]:
rf_data = data.drop(columns = ['Team', 'Date of Moment', 'Transaction Date', 'Buyer', 'Seller', 'Set','Series','Play','Edition State', 'Play ID','Set ID', 'Top Shot Link'])
rf_data.head()

,unique_id,Circulation Count,Serial,Purchase Price,Top Shot Debut,Rookie Premiere,Rookie Mint,Rookie Year,Team_Chicago Bulls,Team_Denver Nuggets,Team_Detroit Pistons,Team_Minnesota Timberwolves,Team_Portland Trail Blazers,Team_Sacramento Kings,Team_Utah Jazz,Buyer_Bullsfan1992,Buyer_dingaling,Seller_Libruary,Seller_NBA_High,Seller_burris_fam,Edition State_CHALLENGE_REWARD_RELEASED,Edition State_VISIBLE,Play ID_05a197a7-4427-4ac0-8115-effbf9e7a5c1,Play ID_3c607722-070f-451a-a53a-b8dd7e098bb6,Play ID_3ca82d8e-a916-47bb-b18b-069bae168e9e,Play ID_636300a8-d470-4dcd-b2f2-3ef623965c74,Play ID_78e08677-cf7a-4911-8126-4c971ef8d4ca,Play ID_7f725955-5bb9-4364-abe4-c9314f000cdb,Play ID_cae9e0eb-edde-4c63-8b0d-ca33c1dc5775,Play ID_f3140cd1-8f9e-4483-853d-9bce3d63180d,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+05a197a7-4427-4ac0-8115-effbf9e7a5c1,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+3c607722-070f-451a-a53a-b8dd7e098bb6,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+3ca82d8e-a916-47bb-b18b-069bae168e9e,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+636300a8-d470-4dcd-b2f2-3ef623965c74,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+78e08677-cf7a-4911-8126-4c971ef8d4ca,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+7f725955-5bb9-4364-abe4-c9314f000cdb,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+cae9e0eb-edde-4c63-8b0d-ca33c1dc5775,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+f3140cd1-8f9e-4483-853d-9bce3d63180d
0,2021-05-17 18:07:47+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,253,265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,2021-05-17 18:05:49+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,228,273,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,2021-05-17 17:58:55+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,283,275,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,2021-05-17 17:50:05+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,359,285,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,2021-05-17 17:45:53+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,79,348,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [63]:
corr_df = rf_data.copy()
corr_df.rename(columns={'Edition State_CHALLENGE_REWARD_RELEASED': 'Challenge Reward'}, inplace=True)
corr_df.head()

,unique_id,Circulation Count,Serial,Purchase Price,Top Shot Debut,Rookie Premiere,Rookie Mint,Rookie Year,Team_Chicago Bulls,Team_Denver Nuggets,Team_Detroit Pistons,Team_Minnesota Timberwolves,Team_Portland Trail Blazers,Team_Sacramento Kings,Team_Utah Jazz,Buyer_Bullsfan1992,Buyer_dingaling,Seller_Libruary,Seller_NBA_High,Seller_burris_fam,Challenge Reward,Edition State_VISIBLE,Play ID_05a197a7-4427-4ac0-8115-effbf9e7a5c1,Play ID_3c607722-070f-451a-a53a-b8dd7e098bb6,Play ID_3ca82d8e-a916-47bb-b18b-069bae168e9e,Play ID_636300a8-d470-4dcd-b2f2-3ef623965c74,Play ID_78e08677-cf7a-4911-8126-4c971ef8d4ca,Play ID_7f725955-5bb9-4364-abe4-c9314f000cdb,Play ID_cae9e0eb-edde-4c63-8b0d-ca33c1dc5775,Play ID_f3140cd1-8f9e-4483-853d-9bce3d63180d,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+05a197a7-4427-4ac0-8115-effbf9e7a5c1,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+3c607722-070f-451a-a53a-b8dd7e098bb6,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+3ca82d8e-a916-47bb-b18b-069bae168e9e,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+636300a8-d470-4dcd-b2f2-3ef623965c74,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+78e08677-cf7a-4911-8126-4c971ef8d4ca,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+7f725955-5bb9-4364-abe4-c9314f000cdb,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+cae9e0eb-edde-4c63-8b0d-ca33c1dc5775,Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+f3140cd1-8f9e-4483-853d-9bce3d63180d
0,2021-05-17 18:07:47+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,253,265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,2021-05-17 18:05:49+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,228,273,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,2021-05-17 17:58:55+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,283,275,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,2021-05-17 17:50:05+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,359,285,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,2021-05-17 17:45:53+00:00f3140cd1-8f9e-4483-853d-9bce3d63180d,382,79,348,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [64]:
corr_plot_df = corr_df[['Purchase Price','Circulation Count', 'Serial','Top Shot Debut', 'Rookie Premiere', 'Rookie Mint', 'Rookie Year', 'Challenge Reward']].corr()['Purchase Price'][:]
corr_plot_df.hvplot.bar(color = 'blue', rot=45)

:Bars   [index]   (Purchase Price)

# RF Model

In [65]:
# Define features set

X = rf_data.copy()
X.drop(columns = ["unique_id","Purchase Price"], axis=1, inplace=True)

In [66]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [67]:
y = rf_data["Purchase Price"].values.reshape(-1, 1)
y[:5]

array([[265],
       [273],
       [275],
       [285],
       [348]])

In [68]:
   
#split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Fit the Standard Scaler with the training data
y_scaler = scaler.fit(y_train)

# Scale the training and testing data
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

#initiate, fit, and train rf_model
rf_model = RandomForestRegressor(n_estimators=400, max_depth=13, max_samples=0.8, random_state=42)
rf_model = rf_model.fit(X_train, y_train)
    
#make predictions for target using X_test
predictions = rf_model.predict(X_test)
    

/var/folders/z6/0v89xnrd4vgcf52w_f36js500000gn/T/ipykernel_69646/4044681673.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model = rf_model.fit(X_train, y_train)


In [69]:
y_train_scaled

array([[-0.26651645],
       [-0.23087885],
       [ 0.07204071],
       ...,
       [-0.31997284],
       [-0.36155004],
       [-0.34967084]])

In [70]:
predictions

array([ 79.3954432 ,  79.35498903,  79.35498903, ...,  65.29012241,
       152.66652199,  79.35498903])

In [71]:
y_pred = rf_model.predict(X_train)
X_train['y_pred']= y_pred

In [72]:
X_train['y_true'] = y_train
true_vs_pred_df = X_train[['y_true','y_pred']]

In [73]:
true_vs_pred_df.hvplot.scatter()

:NdOverlay   [Variable]
   :Scatter   [index]   (value)

In [74]:
importances = rf_model.feature_importances_
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted

[(0.5027046502039186, 'Serial'),
 (0.11124002012742991, 'Circulation Count'),
 (0.10152299747128128, 'Play ID_78e08677-cf7a-4911-8126-4c971ef8d4ca'),
 (0.09795632012719545,
  'Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+78e08677-cf7a-4911-8126-4c971ef8d4ca'),
 (0.02845232277066572, 'Play ID_cae9e0eb-edde-4c63-8b0d-ca33c1dc5775'),
 (0.026055841348696478,
  'Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+cae9e0eb-edde-4c63-8b0d-ca33c1dc5775'),
 (0.016517245991071, 'Seller_Libruary'),
 (0.015484386945313075,
  'Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+636300a8-d470-4dcd-b2f2-3ef623965c74'),
 (0.014432687837290037, 'Play ID_636300a8-d470-4dcd-b2f2-3ef623965c74'),
 (0.01406006230417786,
  'Top Shot Link_https://nbatopshot.com/listings/p2p/757f23fd-f7ae-465c-a006-f09dcfd5dbd5+3ca82d8e-a916-47bb-b18b-069bae168e9e'),
 (0.013741166874521222, 'Play ID_3ca82d8e-a916-47b

In [75]:
# Calc the MSE on actual vs predicted train "y"
mse = mean_squared_error(
        X_train['y_true'],
        X_train['y_pred']
)

# Calc RMSE
rmse = np.sqrt(mse)
rmse

92.53447003495442

In [76]:
# Create New csv with predictions
moment_fv = rf_model.predict(X)

In [76]:
# ADD PREDICTIONS TO ORIGINAL CSV
data['Fair Value'] = moment_fv
data['Player Name'] = player_name
new_csv = data.set_index('Player Name')
new_csv = new_csv.reset_index()
new_csv = new_csv[['Player Name', 'unique_id', 'Fair Value']].set_index('Player Name')
new_csv.to_csv("pred_df.csv")